In [54]:
import numpy as np
import torch
from torch import nn

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim

    def forward(self, encoder_outputs, decoder_hidden):
        # encoder_outputs = [batch size, sequence len, hid dim * directions] = [60,50,1024]
        # decoder_hidden = [num_layers, batch size, hid dim] = [3,60,1024]
        # 一般來說是取 Decoder 最後一層的 hidden state 來做 attention

        decoder_last_hidden = decoder_hidden[-1,:,:]
        decoder_last_hidden = decoder_last_hidden.unsqueeze(1)
        decoder_last_hidden = decoder_last_hidden.transpose(1,2)

        alpha = encoder_outputs.bmm(decoder_last_hidden).squeeze()
        alpha = torch.nn.functional.softmax(alpha,dim=1).unsqueeze(1)
        attention = alpha.bmm(encoder_outputs)
        return attention  # [batch, 1, hid_dim * 2]

In [56]:
hid_dim = 512
batch_size = 60
seq_len = 50
n_layers = 3

encoder_outputs = torch.ones((batch_size,seq_len,hid_dim*2))
decoder_hidden = torch.ones((n_layers,batch_size,hid_dim*2))

atten = Attention(hid_dim)
a = atten(encoder_outputs,decoder_hidden)
a.shape


torch.Size([60, 1, 1024])

In [58]:
a[0:2,0,:]

tensor([[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]])